# ResNet Classifier
As a ResNet Model will be trained. It should help us better understand the performance of the different Yolo Models.

In [1]:
!pip install dotenv
!pip install wandb

In [2]:
from dotenv import load_dotenv
import os
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import wandb
from pathlib import Path
from PIL import Image
from tqdm import tqdm

In [3]:
load_dotenv()
wandb_api_key = os.getenv("WANDB_API_KEY")
print(f"WANDB_API_KEY: [{wandb_api_key[:4]}...]")

WANDB_API_KEY: [69ca...]


In [4]:
wandb.login(key=wandb_api_key)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc
wandb: Currently logged in as: rueedi-tobias (rueedi-tobias-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Sweep Training for Baseline


In [5]:
wandb.init(project="maize_resnet_test", name="resnet_training")

config = wandb.config
config.epochs = 10
config.batch_size = 32
config.lr = 0.001
config.weight_decay = 0.0001
config.img_size = 224
config.num_classes = 2

### Get Data and transform

In [6]:
TRAIN_IMG_DIR = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/train"
VAL_IMG_DIR = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/val"
TRAIN_LABEL_DIR = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/train"
VAL_LABEL_DIR = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/val"

In [33]:
def get_image_level_labels(yolo_label_dir, image_filenames):
    labels = {}
    for img in image_filenames:
        label_file = os.path.splitext(os.path.basename(img))[0] + ".txt"
        label_path = os.path.join(yolo_label_dir, label_file)
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                lines = f.readlines()
                if len(lines) > 0:
                    labels[img] = 1
                else:
                    labels[img] = 0
        else:
            labels[img] = 0
    return labels

transform = transforms.Compose([
    transforms.Resize((config.img_size, config.img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

### Create Dataset

In [26]:
def create_dataset(image_dir, label_dir):
    image_paths = list(Path(image_dir).glob("*.jpg"))
    labels = get_image_level_labels(label_dir, image_paths)

    data = []
    for img_path in image_paths:
        label = labels[img_path]
        data.append((img_path, label))
    return data

class SimpleImageDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label, str(img_path)  # Convert Path object to string


train_data = create_dataset(TRAIN_IMG_DIR, TRAIN_LABEL_DIR)
val_data = create_dataset(VAL_IMG_DIR, VAL_LABEL_DIR)

train_dataset = SimpleImageDataset(train_data, transform=transform)
val_dataset = SimpleImageDataset(val_data, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=8, pin_memory=True)

### Model

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, config.num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)


/opt/conda/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Training 

In [10]:
for epoch in range(config.epochs):
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_acc = train_correct / train_total
    train_loss /= len(train_loader)

    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = val_correct / val_total
    val_loss /= len(val_loader)

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc
    })

    print(f"Epoch {epoch+1}/{config.epochs} - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} - "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

wandb.finish()

Epoch 1/10: 100%|██████████| 340/340 [01:59<00:00,  2.84it/s]


Epoch 1/10 - Train Loss: 0.0018, Train Acc: 0.9999 - Val Loss: 0.0000, Val Acc: 1.0000


Epoch 2/10: 100%|██████████| 340/340 [01:59<00:00,  2.85it/s]


Epoch 2/10 - Train Loss: 0.0007, Train Acc: 0.9998 - Val Loss: 0.0000, Val Acc: 1.0000


Epoch 3/10: 100%|██████████| 340/340 [01:59<00:00,  2.85it/s]


...

Epoch 9/10: 100%|██████████| 340/340 [01:59<00:00,  2.85it/s]


Epoch 9/10 - Train Loss: 0.0001, Train Acc: 1.0000 - Val Loss: 0.0001, Val Acc: 1.0000


Epoch 10/10: 100%|██████████| 340/340 [01:59<00:00,  2.85it/s]


Epoch 10/10 - Train Loss: 0.0001, Train Acc: 1.0000 - Val Loss: 0.0001, Val Acc: 1.0000


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▅▁████████
train_loss,█▄▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁
val_loss,▂▁▄██▇▆▄▄▃
epoch,10
train_accuracy,1
train_loss,5e-05
val_accuracy,1
val_loss,5e-05


### Evaluation

Accuracy is too good. (100%)  
Problem?

In [27]:
train_images = set()
val_images = set()

for batch in train_loader:
    inputs, labels, paths = batch
    train_images.update(paths)

for batch in val_loader:
    inputs, labels, paths = batch
    val_images.update(paths)

overlap = train_images.intersection(val_images)
print(f"Number of overlapping images: {len(overlap)}")
print(f"Overlapping images: {overlap}")


Number of overlapping images: 0
Overlapping images: set()


In [28]:
from collections import Counter

val_labels = []

for _, labels, _ in val_loader:
    val_labels.extend(labels.tolist())

label_counts = Counter(val_labels)
print("Label distribution in validation set:")
for label, count in label_counts.items():
    print(f"Label {label}: {count}")


Label distribution in validation set:
Label 1: 1357


In [29]:
print(f"Trainpicture count: {len(train_images)}")
print(f"Valpcture count: {len(val_images)}")


Anzahl Trainingsbilder: 10858
Anzahl Validierungsbilder: 1357


In [32]:
inputs, labels, path = next(iter(val_loader))
outputs = model(inputs.to(device))
_, predicted = torch.max(outputs, 1)
print(f"Prediction: {predicted}")
print(f"Real Labels: {labels}")


Vorhersagen: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
Tatsächliche Labels: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])


No label with Label 0...